In [13]:
import numpy as np
import random
import math

def move(x, D):
    """Move the state vector ."""
    u = np.random.uniform(-1, 1, len(x))
    alpha = 0.1
    w = 2.1
    R = np.diag(np.dot(D, u))
    D = (1 - alpha) * D + alpha * w * R
    min_step_size = 1e-6 #FIX
    D = np.where(D < min_step_size, min_step_size, D)
    x = x + np.dot(D, u)
    return x, D, u

def objective(x, n):
    """Keane's Bump function.
    n = dimension of the domain
    x = vector in the domain"""
    cos4 = 0
    cos2 = 1
    x_i = 0
    for i in range(n):
        cos4 += math.cos(x[i])**4
        cos2 *= math.cos(x[i])**2
        x_i += (i+1)*(x[i]**2)
    if x_i == 0:
        answer = None
    else:
        answer = np.abs((cos4 - 2*cos2)/math.sqrt(x_i))
    ## we are turning the maximization problem into a minimization problem
    return -1 * answer

def assess_solution(x_prev, n, D_prev, T):
    f_prev = objective(x_prev, n)
    x, D, u = move(x_prev, D_prev)
    ## test constraints and reject move, if necessary
    multiplicative_constraint = 1
    additive_constraint = 0
    for i in range(n):
        multiplicative_constraint *= x[i]
        additive_constraint += x[i]
        if x[i] < 0 or x[i] > 10:
            return x_prev, D_prev
    if multiplicative_constraint <= 0.75 or additive_constraint <= (15*n/2):
        return x_prev, D_prev
    f = objective(x, n)
    if f < f_prev:
        return x, D
    else:
        step_size = np.sqrt(np.sum(np.dot(D, u)**2))
        acceptance_prob = np.exp((f_prev - f)/(T*step_size))
        if random.random() < acceptance_prob:
            return x, D
        else:
            return x_prev, D_prev

def temperature_schedule(T, alpha_T):
    return alpha_T*T

def archive(x, f, archive):
    archive_limit = 10
    if len(archive) < archive_limit:
        archive.append((x, f))
    else:
        for i in range(archive_limit):
            if f < archive[i][1]:
                archive[i] = (x, f)
                break
    return archive

def main(x, n):
    ## initialize covariance matrix
    D = 0.5 * np.identity(n)
    ## initial temperature (FIX)
    T = 1000
    ## final temperature (FIX)
    T_min = 0.1
    ## min acceptance ratio (HYPER PARAMETER)
    min_acceptance_ratio = 0.1
    ## cooling rate (HYPER PARAMETER)
    alpha_T = 0.95
    ## markov chain length (HYPER PARAMETER)
    Lk = 100
    nmin = Lk *0.6

    ## initailize state, covariance matrix
    x_prev = x
    D_prev = D
    ## max number of iterations
    max_iter = 10000
    counter = 0
    ## initialize archive
    archive = []

    while T > T_min:
        acceptances = 0
        ## when either Lk trials or nmin acceptances whichever comes first, decrement the temperature 
        for i in range(Lk):
            x, D = assess_solution(x_prev, n, D_prev, T)
            if not np.array_equal(x, x_prev):
                acceptances += 1
                archive = archive(x, objective(x, n), archive)
            counter += 1
            if acceptances == nmin:
                break
        ## decrement temperature
        T = temperature_schedule(T, alpha_T)
        ## update state, covariance matrix
        x_prev = x
        D_prev = D
        ## no accepted moves in Lk iterations at temperature Tk
        if acceptances/ Lk <= min_acceptance_ratio:
            break
        if counter == max_iter:
            break
    return archive

We need to solve for the initial temperature, the initial solution, and the final temperature. We need to allocate minimum step sizes

In [47]:
import numpy as np
import random
import math

def move(x, D):
    """Move the state vector ."""
    u = np.random.uniform(-1, 1, len(x))
    alpha = 0.1
    w = 2.1
    R = np.diag(np.dot(D, u))
    D = (1 - alpha) * D + alpha * w * R
    min_step_size = 1e-6 #FIX
    D = np.where(D < min_step_size, min_step_size, D)
    x = x + np.dot(D, u)
    return x, D, u

def objective(x, n):
    """Keane's Bump function.
    n = dimension of the domain
    x = vector in the domain"""
    cos4 = 0
    cos2 = 1
    x_i = 0
    for i in range(n):
        cos4 += math.cos(x[i])**4
        cos2 *= math.cos(x[i])**2
        x_i += (i+1)*(x[i]**2)
    if x_i == 0:
        answer = None
    else:
        answer = np.abs((cos4 - 2*cos2)/math.sqrt(x_i))
    ## we are turning the maximization problem into a minimization problem
    return -1 * answer

def assess_solution(x_prev, n, D_prev, T):
    f_prev = objective(x_prev, n)
    x, D, u = move(x_prev, D_prev)
    ## test constraints and reject move, if necessary
    multiplicative_constraint = 1
    additive_constraint = 0
    for i in range(n):
        multiplicative_constraint *= x[i]
        additive_constraint += x[i]
        if x[i] < 0 or x[i] > 10:
            print(f"constraint 0 < x[i] < 10 violated, move rejected")
            return x_prev, D_prev
    if multiplicative_constraint <= 0.75 or additive_constraint >= (15*n/2):
        print(f"mulitplicative constraint or additive constraint violated, move rejected")
        return x_prev, D_prev
    f = objective(x, n)
    if f < f_prev:
        print(f"decreases f, move accepted")
        return x, D
    else:
        step_size = np.sqrt(np.sum(np.dot(D, u)**2))
        acceptance_prob = np.exp((f_prev - f)/(T*step_size))
        if random.random() < acceptance_prob:
            print(f"increases f, BUT move accepted")
            return x, D
        else:
            print(f"increases f, move rejected")
            return x_prev, D_prev

def temperature_schedule(T, alpha_T):
    return alpha_T*T

def archive_function(x, f, archive):
    archive_limit = 10
    if len(archive) < archive_limit:
        archive.append((x, f))
    else:
        for i in range(archive_limit):
            if f < archive[i][1]:
                archive[i] = (x, f)
                break
    return archive

def main(x, n):
    ## initialize covariance matrix
    D = 0.5 * np.identity(n)
    ## initial temperature (FIX)
    T = 1000
    ## final temperature (FIX)
    T_min = 0.1
    ## min acceptance ratio (HYPER PARAMETER)
    min_acceptance_ratio = 0.1
    ## cooling rate (HYPER PARAMETER)
    alpha_T = 0.95
    ## markov chain length (HYPER PARAMETER)
    Lk = 100
    nmin = Lk *0.6

    ## initailize state, covariance matrix
    x_prev = x
    D_prev = D
    ## max number of iterations
    max_iter = 2
    counter = 0
    ## initialize archive
    archive = []

    while T > T_min:
        acceptances = 0
        ## when either Lk trials or nmin acceptances whichever comes first, decrement the temperature 
        for i in range(Lk):
            x, D = assess_solution(x_prev, n, D_prev, T)
            if not np.array_equal(x, x_prev):
                acceptances += 1
                archive = archive_function(x, objective(x, n), archive)
                x_prev = x
                D_prev = D
            counter += 1
            if acceptances == nmin:
                break
        ## decrement temperature
        T = temperature_schedule(T, alpha_T)
        ## update state, covariance matrix
        x_prev = x
        D_prev = D
        print(f"temperature: {T}, acceptances: {acceptances}, counter: {counter}")
        ## no accepted moves in Lk iterations at temperature Tk
        if acceptances/ Lk <= min_acceptance_ratio:
            break
        if counter == max_iter:
            break
    return archive

In [48]:
print(main(np.array([2,2]), 2))

decreases f, move accepted
decreases f, move accepted
increases f, BUT move accepted
decreases f, move accepted
increases f, BUT move accepted
increases f, BUT move accepted
decreases f, move accepted
increases f, BUT move accepted
increases f, BUT move accepted
decreases f, move accepted
decreases f, move accepted
decreases f, move accepted
decreases f, move accepted
increases f, BUT move accepted
decreases f, move accepted
decreases f, move accepted
decreases f, move accepted
decreases f, move accepted
increases f, BUT move accepted
increases f, BUT move accepted
decreases f, move accepted
decreases f, move accepted
decreases f, move accepted
decreases f, move accepted
decreases f, move accepted
decreases f, move accepted
decreases f, move accepted
increases f, BUT move accepted
decreases f, move accepted
increases f, BUT move accepted
increases f, BUT move accepted
increases f, BUT move accepted
increases f, BUT move accepted
increases f, BUT move accepted
decreases f, move accepted